# How to train SentencePiece

- Assume that we have a large collection of bigrams, greater than what we ultimately want to use in our model.
- To train a SentencePiece model, we want to maximize the probability of obtaining a particular tokenization $X = (x_1, x_2, \cdots, x_n)$ of the corpus, given the unigram probabilities $p(x_i), p(x_2), \cdots, p(x_n)$.
- The actual tokenization $X$ is not observed, we only observe the un-tokenized sequence $X$.
- This is a classic problem of maximum likelihood estimation, and we can solve it by the EM algorithm.
- The problem is that thte $x_i$ are all of different lengths, and we cannot apply the EM algorithm directly.
- Instead, we should use a Bayesian approach to solve this problem.


The SentencePiece training objective is given by the following equation:

$$
\mathcal{L}(\theta) = - \sum_{s=1}^{|D|} \log P(X^{(s)}) = - \sum_{s=1}^{|D|} \log (\sum_{\mathbf{x} \in S(\mathbf{x})} P(\mathbf{x}) )
$$

where the $\mathbf{x}$ is a unigram sequence, and $S(\mathbf{x})$ is the set of all possible sequences that can be generated from $\mathbf{x}$.


The steps are as follows:

1. Initialize the unigram probabilities $p(x_i)$. The frequency of each unigram is used as the initial value.
2. M-step: Compute the most likely sequence $\mathbf{x}$ given the current unigram probabilities $p(x_i)$.
3. E-step: Given the current most likely sequence $\mathbf{x}$, update the unigram probabilities $p(x_i)$. In Bayesian setting, the unigram probabilities are defined as:

   $$
   p(x_i | \mathbf{x}) = \frac{c_i}{\sum_{j=1}^{|V|} c_j} \implies \frac{e^{\psi(c_i)}}{\sum_{j=1}^{|V|} e^{\psi(c_j)}} = \frac{e^{\psi(c_i)}}{e^{\psi(\sum_{j=1}^{|V|} c_j)}} = \frac{e^{\psi(c_i)}}{e^{\psi(c_1) + \cdots + \psi(c_n)}}
   $$

   where $c_i$ is the frequency of the unigram $x_i$ in the corpus, $|V|$ is the size of the vocabulary, and $\psi(c_i)$ is the digamma function.

4. Repeat the M-step and E-step until the unigram probabilities converge. The log-likelihood is monotonically increasing, so we can stop the training when the log-likelihood does not increase for a certain number of iterations.


In [1]:
from ekorpkit import eKonf
from ekorpkit.tokenizers.trainers.unigram import UnigramTrainer

sp = UnigramTrainer()

- Initialize the unigram probabilities $p(x_i)$. The frequency of each unigram is used as the initial value.


In [3]:
eKonf.viewsource(sp.initialize_vocab)

    def initialize_vocab(self, words):
        word_freqs = self.get_word_freqs(words)
        sorted_subwords, characters = self.initialize_subwords(word_freqs)

        alphabet = {char: 0 for char in self.initial_alphabet if char not in characters}
        tokens = list(characters.items()) + sorted_subwords + list(alphabet.items())
        tokens = {token: freq for token, freq in tokens}
        tokens = collections.Counter(tokens)
        return tokens, characters



- M-step: Compute the most likely sequence $\mathbf{x}$ given the current unigram probabilities $p(x_i)$.


In [4]:
eKonf.viewsource(sp.M_step)

    def M_step(self, text, trie):
        loss, p = self.forward_step(text, trie)
        tokenization = self.backward_step(text, p)
        return tokenization, loss



- E-step: Given the current most likely sequence $\mathbf{x}$, update the unigram probabilities $p(x_i)$.


In [5]:
eKonf.viewsource(sp.E_step)

    def E_step(self, tokenization, trie):
        # get the new token counts based on updated tokenization
        counts = collections.Counter(tokenization)
        norm = sum(list(counts.values()))

        # we are returning the log probabilties here (alpha=0 prior)
        logsum = digamma(norm)
        for k, v in counts.items():
            counts[k] = digamma(v) - logsum

        for k, v in counts.items():
            trie.set_value(k, v)
        return trie



- Repeat the M-step and E-step until the unigram probabilities converge.


In [6]:
eKonf.viewsource(sp.EM_round)

    def EM_round(self, text, tokens, delta=0.01, n_iterations=10):
        tokenization, old_loss = self.M_step(text, self.trie)
        for step in range(n_iterations):
            print(f"EM iter {step}: ", end="")
            loss, tokenization, trie = self.EM_step(text, tokenization, self.trie)
            print(f"Loss={loss:.2f}")
            if abs(old_loss - loss) < delta:
                break
            old_loss = loss



### Finding the optimal segmentation

- If all of the subwords were of the same length, we could apply the Viterbi algorithm to find the optimal segmentation.
- The Viterbi algorithm is a dynamic programming algorithm that finds the most likely sequence of states, given a sequence of observations and a model of the transition probabilities between states and the emission probabilities of the observations given the states.
  - You have some hidden states $z_1, z_2, \cdots, z_n$, and you want to transition from $z_1 \rightarrow z_2 \rightarrow \cdots \rightarrow z_n$, and you know the transition matrix $A_{ij}$, giving the probability of transitioning from $z_i^{(1)}$ to $z_j^{(2)}$, where $i$ and $j$ are the hidden states, and the superscript indicates the sequence order.
- The problem is that $A$ is not between adjacent states.


- Consider tokenizing the word `hello` given the subwords {`he`, `h`, `ll`, `e`, `o`, `hell`}.
- We can generate the following figure:

  ![](figs/transition.png)

  - Each arrow represents a transition, and the weight of the arrow is the probability of the transition.
  - The goal is to pick arrows that we arrive at `<eos>` (end of sequence) with the highest probability.

- This problem has optimal substructure, so we can apply dynamic programming to solve it.
- For example, assume that we are at the state (4).

  - There are three arrows that can lead to the state (4), a red, a blue, and a green arrow.
  - The highest probability at the state (4) is just the best path from the previous state, plus the probability of the transition.

    $$
    p_i = \max_{j \le i} (p_j p_{j \rightarrow i})
    $$


- The Trie structure is used to find the optimal segmentation.

  - The Trie structure is a tree structure that is used to store a set of strings.
  - The following figure shows the Trie structure for the subwords {`h`, `he`, `hell`, `hello`}:

    ![](figs/trie.png)

  - The root node is the start of the sequence, `<sos>`.
  - Any time we encounter an `end` node, it means that everything in the path from `<sos>` to `end` is a valid subword.
  - The root node `<sos>` will begin with exactly one branch for every unique first character in the vocabulary.
  - As we grow the available subwords, we create more branches.
  - The Trie is going to be the fundamental data structure that the tokenizer uses to store and retrieve the subwords.


In [7]:
eKonf.viewsource(sp.initialize_trie)

    def initialize_trie(self, tokens):
        trie = Trie()
        norm = sum(list(tokens.values()))
        logsum = digamma(norm)

        maxlen = 0
        for tok, val in tokens.items():
            trie.add(tok, digamma(val) - logsum)
            maxlen = max(maxlen, len(tok))

        return trie, maxlen



### Fitting the model

- One of the algorithm for finding the optimal sequence from the Trie is a forwards-backwards algorithm.
  - It is a special subset of the sum-product algorithm for training directed graphical models.
  - More sophisticated algorithms include the Forward-DP Backward-A\* algorithm, and the Forward-Filtering and Backward-Sampling algorithm (FFBS).
- When we compute the forward step, we also store the length of the longest subword that ends at the current position.
- This allows us to backtrack from the end of the sequence to the beginning, and find the optimal segmentation, since the length of the arrow is the length of the subword.
- The EM step puts together the E step and the M step, where the E step is updating the Trie, and the M step is finding the optimal segmentation using the forwards-backwards algorithm explained above.
- Then, fitting the model is just a matter of repeating the EM step until the log-likelihood converges.
- One more thing to consider is to get the desired vocabulary size by pruning the vocabulary.
  - First, prepare more subword tokens than the desired vocabulary size.
  - After each EM step, remove the least probable, say 10%, of the subwords.
  - Repeat this process until the desired vocabulary size is reached.


In [8]:
eKonf.viewsource(sp.forward_step)

    def forward_step(self, text, trie):
        N = len(text)

        # d[i] contains the maximum log_prob of any tokenization
        # of text[:i], initialized to 0 (i.e. log(0)=-infty)
        d = [-np.inf] * (N + 1)

        # p[i] (stands for parent) contains the number of characters of
        # the final token in the most likely sequence that ends at index i
        p = [None] * (N + 1)
        d[0] = 0

        for i in range(1, N + 1):

            # find all possible final words. Have to look back
            # a distance set by the length of the longest token
            for j in range(max(i - self.maxlen, 0), i):

                final_token = text[j:i]
                final_value = trie.get_value(final_token)

                # if the current ending word has a higher log-probability,
                # save that value and store the word (i.e. # chars to backtrack)
                if final_value and d[j] + final_value > d[i]:
                    d[i] = d[j] + final_value
  

In [9]:
eKonf.viewsource(sp.backward_step)

    def backward_step(self, text, p):
        idx = len(p)
        tokenization = []
        while idx > 1:
            # move back the number of steps p tells you to
            next_idx = idx - p[idx - 1]

            # extract the final token
            tok = text[next_idx - 1 : idx - 1]
            tokenization.append(tok)

            idx = next_idx
        tokenization = list(reversed(tokenization))
        return tokenization



### Subword sampling

- To find alternative segmentations, we can save the n-best paths in the forward-backward step.
- Now, we can sample from the n-best paths to find alternative segmentations.


## Implementation


In [2]:
from ekorpkit import eKonf

cfg = eKonf.compose("path")
cfg.cache.uri = "https://github.com/entelecheia/ekorpkit-book/raw/main/assets/data/us_equities_news_sampled.zip"
data = eKonf.load_data("us_equities_news_sampled.parquet", cfg.cached_path)
texts = data.text[:1000]

In [4]:
from ekorpkit.tokenizers.sentencepiece import SentencePieceUnigramTokenizer

texts = data.text[:100]
vocab_size = 5000

sp = SentencePieceUnigramTokenizer()
sp.train_from_iterator(
    texts, vocab_size=vocab_size, min_frequency=10, initial_alphabet=[".", "!", "?"]
)


--- Round 1. Vocab size: 9641 ---
EM iter 0: Loss=-596166.15
EM iter 1: Loss=-593240.93
EM iter 2: Loss=-592453.62
EM iter 3: Loss=-592143.08
EM iter 4: Loss=-592028.75
--- Round 2. Vocab size: 7231 ---
EM iter 0: Loss=-616021.42
EM iter 1: Loss=-615565.28
EM iter 2: Loss=-615379.74
EM iter 3: Loss=-615290.66
EM iter 4: Loss=-615283.06
--- Round 3. Vocab size: 5424 ---
EM iter 0: Loss=-640810.86
EM iter 1: Loss=-640411.09
EM iter 2: Loss=-640241.08
EM iter 3: Loss=-640186.43
EM iter 4: Loss=-640155.71


In [5]:
sp.save("../data/tokenizers", "sp_unigram", pretty=True)

['../data/tokenizers/sp_unigram/vocab.json',
 '../data/tokenizers/sp_unigram/config.json']

In [6]:
from ekorpkit.tokenizers.sentencepiece import SentencePieceUnigramTokenizer

sp = SentencePieceUnigramTokenizer.load("../data/tokenizers", "sp_unigram")

In [7]:
sequence = "Investment opportunities in the company."
print(sp._tokenizer.normalize(sequence))
print(sp._tokenizer.pre_tokenize(sequence))
print(sp._tokenizer.generalized_forward_step("▁company.", sp._tokenizer.trie, 1))

investment opportunities in the company.
['▁Investment', '▁opportunities', '▁in', '▁the', '▁company.']
[None, [1], [2], [3], [4], [5], [6], [7], [8], [1]]


In [8]:
for i in range(5):
    tokenized_text = sp.tokenize(sequence, nbest_size=5)
    print(i, tokenized_text)

0 ['▁', 'investment', '▁opportun', 'iti', 'es', '▁i', 'n', '▁', 'th', 'e', '▁co', 'mpany', '.']
1 ['▁', 'in', 'vestment', '▁opportuni', 'ties', '▁', 'in', '▁', 'th', 'e', '▁', 'company', '.']
2 ['▁', 'investment', '▁o', 'pportuni', 't', 'ies', '▁', 'i', 'n', '▁the', '▁', 'company', '.']
3 ['▁investment', '▁', 'op', 'port', 'un', 'it', 'i', 'es', '▁', 'i', 'n', '▁the', '▁', 'c', 'o', 'mpany', '.']
4 ['▁', 'investmen', 't', '▁', 'opport', 'un', 'itie', 's', '▁in', '▁', 'th', 'e', '▁', 'compan', 'y', '.']


## References

- [SentencePiece](https://github.com/google/sentencepiece)
- [SentencePiece Tokenizer Demystified](https://towardsdatascience.com/sentencepiece-tokenizer-demystified-d0a3aac19b15)
